### Documentation here https://dev.elsevier.com/documentation/ArticleRetrievalAPI.wadl

In [ ]:
# https://api.elsevier.com/content/article/doi/{doi}

In [ ]:
import pandas as pd
import requests
import os
import xmltodict
import json

In [28]:
#read the dataset
df = pd.read_csv("./02_papers_with_abstracts_removed_duplicates_not_filtered_yet.csv")


In [33]:
def downloadFullText(doi, id):
  try:
    # Split the DOI into segments
    doi_segments = doi.split("/")

    # Create the full folder path, including subfolders
    folder_path = os.path.join("./full_texts", *doi_segments)

    # Create the folders if they don't exist
    os.makedirs(folder_path, exist_ok=True)

    # Send the request to the Elsevier API
    response = requests.get(f"https://api.elsevier.com/content/article/doi/{doi}", headers={
      "X-ELS-APIKey": "f4af734d56a079da010b05f250d7c3b5",
        "X-ELS-Insttoken": "7dfb8b168bd5cae0e742f06bfd23fe97",
        "Accept" : "application/json"
    })

    # Check the status code of the response
    if response.status_code == 200:
      json_data = response.json()
      file_name = doi.replace("/", "_")
      file_path = os.path.join(folder_path, f"{file_name}.json")  # Use .json extension
      with open(file_path, 'w') as f:
        f.write(json.dumps(json_data, indent=4))  # Indent for readability

    else:
      # If the request was not successful, print an error message
      print(f"Error retrieving article: {response.text}")
  except Exception as e:
    print(id, str(e))

In [35]:
for i in range(500, len(df)):
    print(str(i)+ "/" + str(len(df)), end="\r")
    with open("last_rec", 'w') as f:
        f.write(str(i))
    doi = df.loc[i]["dc:identifier"].replace("DOI:", "")
    downloadFullText(doi, i)

In [39]:
for i in range(4802, len(df)):
    try:
        print(str(i)+ "/" + str(len(df)), end="\r")
        with open("last_rec", 'w') as f:
            f.write(str(i))
        doi = str(df.loc[i]["dc:identifier"]).replace("DOI:", "")
        downloadFullText(doi, i)
    except:
        print(i, )
        pass